# Song recommender

In [1]:
import requests
import pandas as pd
from time import sleep
import random
from time import sleep
from random import randint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import datetime
import time
import joblib
import numpy as np

secrets_file = open(r"C:\Users\luana\secrets.txt","r")
string = secrets_file.read()

secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['secret']))

In [2]:
scaler = joblib.load('scaler.save')
model = joblib.load('model.save')

## Run these lines to update hot songs

In [3]:
# I want the hot songs to be always up to date, so I will use the functions to scrape them from Spotify

def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(random.uniform(0.5,1))
        
    #creating df with needed columns
    playlist_df = pd.DataFrame(columns=['song','artist','popularity','danceability','energy','key','loudness','mode',
                                    'speechiness','acousticness','instrumentalness','liveness','valence','tempo',
                                    'type','id','uri','track_href','analysis_url','duration_ms','time_signature'])
    
    for i in range(len(tracks)):
        sleep(random.uniform(0.5,1))
        
        try:
        
            song_id = tracks[i]['track']['id']

            # Song name, artists and popularity obtained directly with request

            song = sp.track(song_id)['name']

            # Getting up to three different artists for each song
            artist_1 = sp.track(song_id)['artists'][0]['name']
            artist_2 = ''
            try:
                artist_2 = sp.track(song_id)['artists'][1]['name']
                artist_2= " & "+artist_2
            except:
                pass
            artist_3 = ''
            try:
                artist_3 = sp.track(song_id)['artists'][2]['name']
                artist_3= " & "+artist_3
            except:
                pass
            artists = (artist_1+artist_2+artist_3)

            popularity = sp.track(song_id)['popularity']

            track_dict = {'song':song, 'artist':artists, 'popularity':popularity}
            df_track = pd.DataFrame([track_dict])

            # Now the audio features
            features_dict = sp.audio_features(song_id)
            df_features = pd.DataFrame(features_dict)

            # Complete df for one song
            song_df = pd.concat([df_track, df_features], axis=1)

            # Complete df with all songs
            playlist_df = pd.concat([playlist_df, song_df],axis=0)

            
        except:
            pass    

    return playlist_df

In [4]:
def search_export_playlists(queries_list):
    songs_df = pd.DataFrame(columns=['song','artist','popularity','danceability','energy','key','loudness','mode',
                                    'speechiness','acousticness','instrumentalness','liveness','valence','tempo',
                                    'type','id','uri','track_href','analysis_url','duration_ms','time_signature'])
    
    for i in range (len(queries_list)):
        sleep(random.uniform(0.5,1))
        try:
                     
            results = sp.search(q=queries_list[i], type='playlist', limit=1)
            playlist_id = results['playlists']['items'][0]['id']

            playlist_df = get_playlist_tracks(playlist_id)

            songs_df = pd.concat([songs_df, playlist_df], axis=0)

            print(queries_list[i], ' done')
 
        except:
            pass
        
   
    return songs_df

In [5]:
queries = ['Billboard Hot 100', 'Top 100 Global']

hot_songs = search_export_playlists(queries)
hot_songs = hot_songs.drop_duplicates(['uri']).reset_index(drop=True)
hot_songs

Billboard Hot 100  done
Top 100 Global  done


,song,artist,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Bad Habit,Steve Lacy,93,0.686,0.507,1,-7.097,1,0.0357,0.626,...,0.383,0.69,168.955,audio_features,5CM4UuQ9Gnd6K2YyKGPMoK,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK,https://api.spotify.com/v1/tracks/5CM4UuQ9Gnd6...,https://api.spotify.com/v1/audio-analysis/5CM4...,232115,4
1,Unholy (feat. Kim Petras),Sam Smith & Kim Petras,100,0.714,0.472,2,-7.375,1,0.0864,0.013,...,0.266,0.238,131.121,audio_features,3nqQXoyQOWXiESFLlDF1hG,spotify:track:3nqQXoyQOWXiESFLlDF1hG,https://api.spotify.com/v1/tracks/3nqQXoyQOWXi...,https://api.spotify.com/v1/audio-analysis/3nqQ...,156943,4
2,As It Was,Harry Styles,96,0.52,0.731,6,-5.338,0,0.0557,0.342,...,0.311,0.662,173.93,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
3,I Like You (A Happier Song) (with Doja Cat),Post Malone & Doja Cat,92,0.733,0.67,5,-6.009,1,0.0751,0.121,...,0.121,0.472,100.964,audio_features,0O6u0VJ46W86TxN9wgyqDj,spotify:track:0O6u0VJ46W86TxN9wgyqDj,https://api.spotify.com/v1/tracks/0O6u0VJ46W86...,https://api.spotify.com/v1/audio-analysis/0O6u...,192841,4
4,Sunroof,Nicky Youre & dazy,93,0.768,0.714,10,-5.11,1,0.0401,0.352,...,0.15,0.842,131.443,audio_features,4h4QlmocP3IuwYEj2j14p8,spotify:track:4h4QlmocP3IuwYEj2j14p8,https://api.spotify.com/v1/tracks/4h4QlmocP3Iu...,https://api.spotify.com/v1/audio-analysis/4h4Q...,163026,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Freaky Deaky,Tyga & Doja Cat,77,0.799,0.653,2,-5.453,0,0.037,0.019,...,0.0935,0.236,104.019,audio_features,3vySEUpD0tc801F2WZDLYw,spotify:track:3vySEUpD0tc801F2WZDLYw,https://api.spotify.com/v1/tracks/3vySEUpD0tc8...,https://api.spotify.com/v1/audio-analysis/3vyS...,215281,4
164,Shape of You,Ed Sheeran,86,0.825,0.652,1,-3.183,0,0.0802,0.581,...,0.0931,0.931,95.977,audio_features,7qiZfU4dY1lWllzX7mPBI3,spotify:track:7qiZfU4dY1lWllzX7mPBI3,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,https://api.spotify.com/v1/audio-analysis/7qiZ...,233713,4
165,Woah,Lil Baby,69,0.882,0.602,11,-5.554,1,0.245,0.0177,...,0.178,0.43,142.959,audio_features,598yNsG1JFrGd4n2kMoz7F,spotify:track:598yNsG1JFrGd4n2kMoz7F,https://api.spotify.com/v1/tracks/598yNsG1JFrG...,https://api.spotify.com/v1/audio-analysis/598y...,183011,4
166,Wants and Needs (feat. Lil Baby),Drake & Lil Baby,85,0.578,0.449,1,-6.349,1,0.286,0.0618,...,0.119,0.1,136.006,audio_features,65OVbaJR5O1RmwOQx0875b,spotify:track:65OVbaJR5O1RmwOQx0875b,https://api.spotify.com/v1/tracks/65OVbaJR5O1R...,https://api.spotify.com/v1/audio-analysis/65OV...,192956,4


In [7]:
hot_songs.to_csv(r'C:\Users\luana\IH-Labs\lab-web-scraping-multiple-pages\hot_songs.csv', index=False)

In [20]:
hot_songs = pd.read_csv(r'C:\Users\luana\IH-Labs\lab-web-scraping-multiple-pages\hot_songs.csv')
hot_songs

,song,artist,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Bad Habit,Steve Lacy,93,0.686,0.507,1,-7.097,1,0.0357,0.6260,...,0.3830,0.690,168.955,audio_features,5CM4UuQ9Gnd6K2YyKGPMoK,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK,https://api.spotify.com/v1/tracks/5CM4UuQ9Gnd6...,https://api.spotify.com/v1/audio-analysis/5CM4...,232115,4
1,Unholy (feat. Kim Petras),Sam Smith & Kim Petras,100,0.714,0.472,2,-7.375,1,0.0864,0.0130,...,0.2660,0.238,131.121,audio_features,3nqQXoyQOWXiESFLlDF1hG,spotify:track:3nqQXoyQOWXiESFLlDF1hG,https://api.spotify.com/v1/tracks/3nqQXoyQOWXi...,https://api.spotify.com/v1/audio-analysis/3nqQ...,156943,4
2,As It Was,Harry Styles,96,0.520,0.731,6,-5.338,0,0.0557,0.3420,...,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
3,I Like You (A Happier Song) (with Doja Cat),Post Malone & Doja Cat,92,0.733,0.670,5,-6.009,1,0.0751,0.1210,...,0.1210,0.472,100.964,audio_features,0O6u0VJ46W86TxN9wgyqDj,spotify:track:0O6u0VJ46W86TxN9wgyqDj,https://api.spotify.com/v1/tracks/0O6u0VJ46W86...,https://api.spotify.com/v1/audio-analysis/0O6u...,192841,4
4,Sunroof,Nicky Youre & dazy,93,0.768,0.714,10,-5.110,1,0.0401,0.3520,...,0.1500,0.842,131.443,audio_features,4h4QlmocP3IuwYEj2j14p8,spotify:track:4h4QlmocP3IuwYEj2j14p8,https://api.spotify.com/v1/tracks/4h4QlmocP3Iu...,https://api.spotify.com/v1/audio-analysis/4h4Q...,163026,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Freaky Deaky,Tyga & Doja Cat,77,0.799,0.653,2,-5.453,0,0.0370,0.0190,...,0.0935,0.236,104.019,audio_features,3vySEUpD0tc801F2WZDLYw,spotify:track:3vySEUpD0tc801F2WZDLYw,https://api.spotify.com/v1/tracks/3vySEUpD0tc8...,https://api.spotify.com/v1/audio-analysis/3vyS...,215281,4
164,Shape of You,Ed Sheeran,86,0.825,0.652,1,-3.183,0,0.0802,0.5810,...,0.0931,0.931,95.977,audio_features,7qiZfU4dY1lWllzX7mPBI3,spotify:track:7qiZfU4dY1lWllzX7mPBI3,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,https://api.spotify.com/v1/audio-analysis/7qiZ...,233713,4
165,Woah,Lil Baby,69,0.882,0.602,11,-5.554,1,0.2450,0.0177,...,0.1780,0.430,142.959,audio_features,598yNsG1JFrGd4n2kMoz7F,spotify:track:598yNsG1JFrGd4n2kMoz7F,https://api.spotify.com/v1/tracks/598yNsG1JFrG...,https://api.spotify.com/v1/audio-analysis/598y...,183011,4
166,Wants and Needs (feat. Lil Baby),Drake & Lil Baby,85,0.578,0.449,1,-6.349,1,0.2860,0.0618,...,0.1190,0.100,136.006,audio_features,65OVbaJR5O1RmwOQx0875b,spotify:track:65OVbaJR5O1RmwOQx0875b,https://api.spotify.com/v1/tracks/65OVbaJR5O1R...,https://api.spotify.com/v1/audio-analysis/65OV...,192956,4


## Get clustered songs

In [9]:
songs_clustered = pd.read_csv(r'C:\Users\luana\IH-Labs\lab-web-scraping-multiple-pages\songs_clustered.csv')

## Song recommender

In [15]:
# Function to get input and find spotify info
def song_inf(song_id):
    song_df = pd.DataFrame(columns=['danceability','energy','key','loudness','mode',
                                    'speechiness','acousticness','instrumentalness','liveness','valence','tempo',
                                    'type','id','uri','track_href','analysis_url','duration_ms','time_signature'])

    features_dict = sp.audio_features(song_id)
    song_df = pd.DataFrame(features_dict)
    song_df = song_df.select_dtypes(np.number)
   
    return song_df

In [16]:
def song_recommender():
    user_song = str(input('Please enter the name of a song you like\n')).lower()
    results = sp.search(q=user_song, type='track', limit=20)
    answer = ''
    n = 0
    
    try:        
        while answer != 'y' and n<21:
            message = ('Did you mean '+results["tracks"]["items"][n]["name"]+' by '
                       +results["tracks"]["items"][n]["artists"][0]["name"]+'?\n'+'Type y for yes or n for No\n')
            song_id = results["tracks"]["items"][n]["id"]
            answer = str(input(message).lower()[:1])
            n = n+1
    except:
        pass

    if answer == 'n':       
        print('Sorry, we cannot find your song. Please try another one.')

    elif answer == 'y':
        if song_id in hot_songs['id'].tolist():
            i = random.randint(0,len(hot_songs))
            print('You may also like',hot_songs['song'][i].title(), "by",hot_songs['artist'][i].title())

        else:
            song_df = song_inf(song_id)
            cluster=model.predict(pd.DataFrame(scaler.transform(song_df), columns=song_df.columns))
            cluster = cluster[0]
            cluster_df = songs_clustered[songs_clustered['cluster']==cluster].reset_index(drop=True)
            i = random.randint(0,len(cluster_df))
            cluster_df['song'][i]
            suggestion = 'We think you may also like '+cluster_df['song'][i]+' by '+cluster_df['artist'][i]
            print(suggestion)

    else:
        print('Invalid answer. Please start again')


In [17]:
song_recommender()

Please enter the name of a song you like
da da da
Did you mean DA DA DANCE by BABYMETAL?
Type y for yes or n for No
n
Did you mean Sunroof by Nicky Youre?
Type y for yes or n for No
n
Did you mean Dah Dah DahDah by Nardo Wick?
Type y for yes or n for No
n
Did you mean Da Da Da by Catchybeatz?
Type y for yes or n for No
n
Did you mean Young Dumb & Broke by Khalid?
Type y for yes or n for No
n
Did you mean Da Da Da I Don't Love You You Don't Love Me Aha Aha Aha by Trio?
Type y for yes or n for No
y
We think you may also like Wake Me Up Before You Go-Go by Wham!


In [18]:
song_recommender()

Please enter the name of a song you like
bohemian rapsody
Did you mean Bohemian Rhapsody - Remastered 2011 by Queen?
Type y for yes or n for No
y
We think you may also like Angie by The Rolling Stones


In [19]:
song_recommender()

Please enter the name of a song you like
the real slim shady
Did you mean The Real Slim Shady by Eminem?
Type y for yes or n for No
y
We think you may also like I Wish by Stevie Wonder


In [17]:
song_recommender()

Please enter the name of a song you like
as it was
Did you mean As It Was by Harry Styles?
Type y for yes or n for No
y
You may also like Pushin P (Feat. Young Thug) by Gunna & Future & Young Thug


In [21]:
song_recommender()

Please enter the name of a song you like
song not listed
Did you mean Cadabra Freestyle by Abra Cadabra?
Type y for yes or n for No
n
Did you mean Hanging On by Blue Water Highway?
Type y for yes or n for No
n
Did you mean Cadabra Freestyle by Abra Cadabra?
Type y for yes or n for No
n
Did you mean Gone by Ocean Wisdom?
Type y for yes or n for No
n
Did you mean Peace Inside by Amery Rey Tuesta?
Type y for yes or n for No
n
Did you mean K.O.T. by P1no?
Type y for yes or n for No
n
Did you mean Honoring Song by not listed?
Type y for yes or n for No
n
Did you mean Not An Emergency by Listed Class A?
Type y for yes or n for No
n
Sorry, we cannot find your song. Please try another one.
